In [1]:
# PHASE 1: SETUP AND DATA LOADING

import os
import sys
import json
import subprocess
import time
import glob
from pathlib import Path
from datetime import datetime
from collections import defaultdict


print("PHASE 1: SETUP AND DATA LOADING")
print()

WORK_DIR = os.getcwd()
DATA_ROOT = "."
ANNOTATIONS_DIR = f"{DATA_ROOT}/annotations"
IMAGES_DIR = f"{DATA_ROOT}/images"

print("Installing dependencies...")

from ultralytics import YOLO
import cv2
import numpy as np
import torch

print("Dependencies installed successfully")
print()
print(f"Working directory: {WORK_DIR}")
print(f"Data root: {DATA_ROOT}")
print()

print("Loading dataset JSON files...")

try:
    with open(f"{ANNOTATIONS_DIR}/instances.json", 'r') as f:
        coco_data = json.load(f)
    print("Loaded instances.json")
except Exception as e:
    print(f"Error loading instances.json: {e}")
    raise

try:
    with open(f"{ANNOTATIONS_DIR}/instances_train.json", 'r') as f:
        train_data = json.load(f)
    print("Loaded instances_train.json")
except Exception as e:
    print(f"Error loading instances_train.json: {e}")
    raise

try:
    with open(f"{ANNOTATIONS_DIR}/instances_val.json", 'r') as f:
        val_data = json.load(f)
    print("Loaded instances_val.json")
except Exception as e:
    print(f"Error loading instances_val.json: {e}")
    raise

print()
print("Creating dataset mappings...")

train_img_ids = {img['id'] for img in train_data['images']}
val_img_ids = {img['id'] for img in val_data['images']}
images_dict = {img['id']: img for img in coco_data['images']}
categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
annotations = coco_data['annotations']

class_names = sorted([categories[c] for c in categories.keys()])

print()
print("Dataset Statistics:")
print(f"  Training images: {len(train_img_ids)}")
print(f"  Validation images: {len(val_img_ids)}")
print(f"  Total images: {len(images_dict)}")
print(f"  Total annotations: {len(annotations)}")
print(f"  Number of classes: {len(class_names)}")
print(f"  Classes: {class_names}")
print()

print("Verifying image files...")
missing_count = 0
for img_id, img_info in list(images_dict.items())[:5]:
    img_path = f"{IMAGES_DIR}/{img_info['file_name']}"
    if not os.path.exists(img_path):
        missing_count += 1
        print(f"Missing: {img_path}")

if missing_count == 0:
    print("All checked images exist")
print()

print("Setup and Data Loading Completed!")
print()

PHASE 1: SETUP AND DATA LOADING

Installing dependencies...
Dependencies installed successfully

Working directory: /courses/CS5330.202610/students/shinkar.k/Final Project
Data root: .

Loading dataset JSON files...
Loaded instances.json
Loaded instances_train.json
Loaded instances_val.json

Creating dataset mappings...

Dataset Statistics:
  Training images: 393
  Validation images: 107
  Total images: 500
  Total annotations: 9787
  Number of classes: 3
  Classes: ['human', 'trafficsign', 'vehicle']

Verifying image files...
All checked images exist

Setup and Data Loading Completed!



In [2]:
# PHASE 2: CONVERT TO YOLO FORMAT

print("PHASE 2: CONVERT TO YOLO FORMAT")
print()

import shutil

print("Creating directory structure...")

os.makedirs(f"{WORK_DIR}/dataset/images/train", exist_ok=True)
os.makedirs(f"{WORK_DIR}/dataset/images/val", exist_ok=True)
os.makedirs(f"{WORK_DIR}/dataset/labels/train", exist_ok=True)
os.makedirs(f"{WORK_DIR}/dataset/labels/val", exist_ok=True)

print("Directories created")
print()

def create_yolo_annotations(img_ids, split_name):
    processed = 0
    skipped = 0
    
    for img_id in img_ids:
        img_info = images_dict[img_id]
        img_path = f"{IMAGES_DIR}/{img_info['file_name']}"
        
        if not os.path.exists(img_path):
            skipped += 1
            continue
        
        width, height = img_info['width'], img_info['height']
        
        img_annotations = [ann for ann in annotations if ann['image_id'] == img_id]
        
        txt_filename = Path(img_info['file_name']).stem + ".txt"
        txt_path = f"{WORK_DIR}/dataset/labels/{split_name}/{txt_filename}"
        
        with open(txt_path, 'w') as f:
            for ann in img_annotations:
                class_id = ann['category_id'] - 1
                bbox = ann['bbox']
                
                x_center = (bbox[0] + bbox[2] / 2) / width
                y_center = (bbox[1] + bbox[3] / 2) / height
                w_norm = bbox[2] / width
                h_norm = bbox[3] / height
                
                f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")
        
        dst_img_path = f"{WORK_DIR}/dataset/images/{split_name}/{Path(img_info['file_name']).name}"
        if not os.path.exists(dst_img_path):
            shutil.copy(img_path, dst_img_path)
        
        processed += 1
    
    return processed, skipped

print("Converting training set...")
train_processed, train_skipped = create_yolo_annotations(train_img_ids, "train")
print(f"Training: {train_processed} processed, {train_skipped} skipped")
print()

print("Converting validation set...")
val_processed, val_skipped = create_yolo_annotations(val_img_ids, "val")
print(f"Validation: {val_processed} processed, {val_skipped} skipped")
print()

print("Creating YOLO data.yaml...")

yaml_content = f"""path: {WORK_DIR}/dataset
train: images/train
val: images/val

nc: {len(class_names)}
names: {class_names}
"""

with open(f"{WORK_DIR}/dataset/data.yaml", 'w') as f:
    f.write(yaml_content)

print("Created data.yaml")
print()

print("Converting to YOLO format completed!")
print()


PHASE 2: CONVERT TO YOLO FORMAT

Creating directory structure...
Directories created

Converting training set...
Training: 393 processed, 0 skipped

Converting validation set...
Validation: 107 processed, 0 skipped

Creating YOLO data.yaml...
Created data.yaml

Converting to YOLO format completed!



In [3]:
# PHASE 3: TRAIN MODEL WITH AUGMENTATION FOR IMPROVED RECALL

print("PHASE 3: TRAINING THE MODEL WITH AUGMENTATION FOR IMPROVED RECALL")
print()

print("Loading YOLOv8s model...")
model = YOLO('yolov8s.pt')
print("Model loaded")
print()

print("Checking available compute devices...")
print("NOTE: For this environment, forcing CPU to avoid CUDA compatibility issues.")
cuda_available = False
device_str = "cpu"
print(f"CUDA available (ignored): {torch.cuda.is_available()}")
print(f"Using device: {device_str}")
print()

batch_size = 8
print("Running on CPU - batch size set to 8")
print()

print("Starting fine tuning")
if not cuda_available:
    print("Training on CPU")
else:
    print("Training on GPU")
print()

start_train = time.time()

results = model.train(
    data=f"{WORK_DIR}/dataset/data.yaml",
    epochs=50,
    imgsz=640,
    batch=batch_size,
    patience=15,
    device=device_str,
    project=f"{WORK_DIR}/runs",
    name="survue_yolov8n_optimized",
    save=True,
    verbose=False,
    workers=4,
    augment=True,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10,
    translate=0.1,
    scale=0.5,
    flipud=0.5,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1,
    lrf=0.01
)

train_time = time.time() - start_train

print()
print(f"Training complete in {train_time/60:.2f} minutes")
print()

best_model_path = f"{WORK_DIR}/runs/survue_yolov8n_optimized/weights/best.pt"
if os.path.exists(best_model_path):
    model_size = os.path.getsize(best_model_path) / (1024 * 1024)
    print(f"Best model saved: {best_model_path}")
    print(f"Model size: {model_size:.2f} MB")
else:
    print(f"Error: Best model not found at {best_model_path}")
print()

print("Model Training Completed!")
print()

PHASE 3: TRAINING THE MODEL WITH AUGMENTATION FOR IMPROVED RECALL

Loading YOLOv8s model...
Model loaded

Checking available compute devices...
NOTE: For this environment, forcing CPU to avoid CUDA compatibility issues.
CUDA available (ignored): True
Using device: cpu

Running on CPU - batch size set to 8

Starting fine tuning
Training on CPU

Ultralytics 8.3.234 🚀 Python-3.12.9 torch-2.9.1+cu128 CPU (Intel Xeon CPU E5-2680 v4 @ 2.40GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/courses/CS5330.202610/students/shinkar.k/Final Project/dataset/data.yaml, degrees=10, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=Non

In [6]:
# PHASE 4: EVALUATE ON VALIDATION SET

print("PHASE 4: EVALUATE ON VALIDATION SET")
print()

import glob
run_dirs = glob.glob(f"{WORK_DIR}/runs/survue_yolov8n_optimized*")
run_dirs.sort(key=os.path.getmtime, reverse=True)
best_model_path = f"{run_dirs[0]}/weights/best.pt"
model = YOLO(best_model_path)
print(f"Loaded best model from {best_model_path}")
print()

print("Running inference on validation images...")
print("Using confidence threshold: 0.15 (optimized for recall)")
print()

val_image_ids = list(val_img_ids)
detections_by_image = defaultdict(list)
ground_truth_by_image = defaultdict(list)

os.makedirs(f"{WORK_DIR}/results/predictions", exist_ok=True)

start_inference = time.time()

for idx, img_id in enumerate(val_image_ids):
    if idx % 50 == 0 or idx == len(val_image_ids) - 1:
        print(f"  Processing: {idx+1}/{len(val_image_ids)}")
    
    img_info = images_dict[img_id]
    img_path = f"{IMAGES_DIR}/{img_info['file_name']}"
    
    if not os.path.exists(img_path):
        continue
    
    width, height = img_info['width'], img_info['height']
    
    img_gt = [ann for ann in annotations if ann['image_id'] == img_id]
    for ann in img_gt:
        bbox = ann['bbox']
        ground_truth_by_image[img_id].append({
            'class_id': ann['category_id'] - 1,
            'bbox': [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]],
            'area': ann['area']
        })
    
    results = model.predict(source=img_path, conf=0.15, device=device_str, verbose=False)
    result = results[0]
    
    if result.boxes is not None:
        boxes = result.boxes
        for i in range(len(boxes)):
            box = boxes.xyxy[i].cpu().numpy()
            conf = float(boxes.conf[i])
            class_id = int(boxes.cls[i])
            
            detections_by_image[img_id].append({
                'class_id': class_id,
                'bbox': box.tolist(),
                'confidence': conf
            })
    
    if idx % 10 == 0:
        annotated = result.plot()
        output_path = f"{WORK_DIR}/results/predictions/{idx:04d}_pred.jpg"
        cv2.imwrite(output_path, annotated)

inference_time = time.time() - start_inference
avg_inference_time = inference_time / len(val_image_ids)

print()
print(f"Inference complete")
print(f"Total time: {inference_time:.2f}s")
print(f"Average per image: {avg_inference_time:.4f}s ({1/avg_inference_time:.2f} FPS)")
print()

print("Evaluation on Validation set Complete")
print()

PHASE 4: EVALUATE ON VALIDATION SET

Loaded best model from /courses/CS5330.202610/students/shinkar.k/Final Project/runs/survue_yolov8n_optimized6/weights/best.pt

Running inference on validation images...
Using confidence threshold: 0.15 (optimized for recall)

  Processing: 1/107
  Processing: 51/107
  Processing: 101/107
  Processing: 107/107

Inference complete
Total time: 8.10s
Average per image: 0.0757s (13.20 FPS)

Evaluation on Validation set Complete



In [19]:
# PHASE 5: CALCULATE METRICS (YOLOv8s ONLY)

print("PHASE 5: CALCULATE METRICS (YOLOv8s)")
print()

from statistics import mean
import glob

# ---------- IoU + custom metrics ----------

def calculate_iou(box1, box2):
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])

    if x2_inter <= x1_inter or y2_inter <= y1_inter:
        return 0.0

    inter_area = (x2_inter - x1_inter) * (y2_inter - y1_inter)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area > 0 else 0.0


def calculate_metrics_at_iou(detections_by_image, ground_truth_by_image, iou_threshold=0.5):
    tp = 0
    fp = 0
    fn = 0
    tp_ious = []

    for img_id in ground_truth_by_image:
        gt = ground_truth_by_image[img_id]
        dets = detections_by_image.get(img_id, [])

        # sort detections by confidence (high → low)
        sorted_dets = sorted(dets, key=lambda x: x["confidence"], reverse=True)
        gt_matched = [False] * len(gt)

        for det in sorted_dets:
            max_iou = 0.0
            max_gt_idx = -1

            for gt_idx, gt_box in enumerate(gt):
                if gt_matched[gt_idx]:
                    continue

                iou = calculate_iou(det["bbox"], gt_box["bbox"])
                if iou > max_iou:
                    max_iou = iou
                    max_gt_idx = gt_idx

            if max_iou >= iou_threshold and max_gt_idx >= 0:
                tp += 1
                gt_matched[max_gt_idx] = True
                tp_ious.append(max_iou)
            else:
                fp += 1

        fn += sum(1 for matched in gt_matched if not matched)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
    mean_iou = mean(tp_ious) if tp_ious else 0.0

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "tp": tp,
        "fp": fp,
        "fn": fn,
        "mean_iou": mean_iou,
    }


print("Calculating custom metrics at IoU=0.50 and IoU=0.75...")
metrics_50 = calculate_metrics_at_iou(detections_by_image, ground_truth_by_image, iou_threshold=0.5)
metrics_75 = calculate_metrics_at_iou(detections_by_image, ground_truth_by_image, iou_threshold=0.75)

total_gt = sum(len(gt) for gt in ground_truth_by_image.values())
total_detections = sum(len(dets) for dets in detections_by_image.values())

print()
print("CUSTOM METRICS (from Phase 4 detections)")
print("----------------------------------------")
print(f"IoU=0.50 -> P: {metrics_50['precision']:.4f}, R: {metrics_50['recall']:.4f}, "
      f"F1: {metrics_50['f1']:.4f}, mean IoU(TP): {metrics_50['mean_iou']:.4f}")
print(f"IoU=0.75 -> P: {metrics_75['precision']:.4f}, R: {metrics_75['recall']:.4f}, "
      f"F1: {metrics_75['f1']:.4f}, mean IoU(TP): {metrics_75['mean_iou']:.4f}")
print()
print(f"True Positives (IoU=0.50): {metrics_50['tp']}")
print(f"False Positives (IoU=0.50): {metrics_50['fp']}")
print(f"False Negatives (IoU=0.50): {metrics_50['fn']}")
print(f"Total detections:          {total_detections}")
print(f"Total ground-truth boxes:  {total_gt}")
print()

# ---------- Official YOLOv8 metrics (mAP) ----------

print("Running official YOLOv8 validation on the val split for mAP...")
yolo_val_metrics = model.val(
    data=f"{WORK_DIR}/dataset/data.yaml",
    split="val",
    imgsz=640,
    batch=batch_size,
    device=device_str,
    verbose=False
)

# YOLO's MetricResults object
map50 = float(yolo_val_metrics.box.map50)   # mAP@0.50
map5095 = float(yolo_val_metrics.box.map)   # mAP@0.50:0.95

print()
print("YOLOv8 OFFICIAL METRICS")
print("------------------------")
print(f"mAP@0.50:        {map50:.4f}")
print(f"mAP@0.50:0.95:   {map5095:.4f}")
print()

# ---------- Model size & speed (safe, no crash if best.pt missing) ----------

print("Checking model size (if best.pt is available)...")

best_model_path = None
model_size_mb = None

best_candidates = glob.glob(f"{WORK_DIR}/runs/*/weights/best.pt")
if best_candidates:
    # pick the most recent best.pt
    best_model_path = max(best_candidates, key=os.path.getmtime)
    model_size_mb = os.path.getsize(best_model_path) / (1024 * 1024)
    print(f"Found best.pt at: {best_model_path}")
    print(f"Model size:       {model_size_mb:.2f} MB")
else:
    print("No best.pt found under runs/*/weights/. Skipping model-size calculation.")

print()
print("RUNTIME / HARDWARE SUMMARY")
print("--------------------------")

if model_size_mb is not None:
    print(f"Model Size:          {model_size_mb:.2f} MB")
else:
    print("Model Size:          unknown (best.pt not found)")

if 'avg_inference_time' in globals():
    print(f"Inference Time:      {avg_inference_time:.4f}s per image "
          f"({1/avg_inference_time:.2f} FPS)")
else:
    print("Inference Time:      (avg_inference_time not defined – run Phase 4 first)")

# simple suitability flags (only if we know size & speed)
lightweight = model_size_mb is not None and model_size_mb < 50
fast = 'avg_inference_time' in globals() and avg_inference_time < 0.1
accurate = (metrics_50['precision'] > 0.7) and (metrics_50['recall'] > 0.7)

print()
print(f"Lightweight (<50MB):       {lightweight}")
if 'avg_inference_time' in globals():
    print(f"Fast (<0.1s / image):      {fast}")
print(f"Accurate (P,R>0.7 @0.5IoU): {accurate}")
print()

# ---------- Save everything to JSON ----------

print("Saving metrics to JSON...")

metrics_data = {
    "timestamp": datetime.now().isoformat(),
    "model_info": {
        "architecture": "YOLOv8s",
        "training_approach": "Fine-tuned with augmentation on Survue dataset",
        "model_size_mb": float(model_size_mb) if model_size_mb is not None else None,
        "inference_time_per_image": float(avg_inference_time) if 'avg_inference_time' in globals() else None,
        "fps": float(1 / avg_inference_time) if 'avg_inference_time' in globals() else None,
    },
    "training_config": {
        "epochs": 50,
        "batch_size": batch_size,
        "image_size": 640,
        "patience": 15,
        "device": device_str,
        "augmentation": {
            "hsv_h": 0.015,
            "hsv_s": 0.7,
            "hsv_v": 0.4,
            "degrees": 10,
            "translate": 0.1,
            "scale": 0.5,
            "flipud": 0.5,
            "fliplr": 0.5,
            "mosaic": 1.0,
            "mixup": 0.1,
        },
    },
    "dataset": {
        "training_images": int(train_processed),
        "validation_images": int(val_processed),
        "classes": class_names,
        "total_annotations": int(total_gt),
    },
    "custom_metrics": {
        "iou_0_50": {
            "precision": float(metrics_50["precision"]),
            "recall": float(metrics_50["recall"]),
            "f1_score": float(metrics_50["f1"]),
            "mean_iou_tp": float(metrics_50["mean_iou"]),
            "tp": int(metrics_50["tp"]),
            "fp": int(metrics_50["fp"]),
            "fn": int(metrics_50["fn"]),
        },
        "iou_0_75": {
            "precision": float(metrics_75["precision"]),
            "recall": float(metrics_75["recall"]),
            "f1_score": float(metrics_75["f1"]),
            "mean_iou_tp": float(metrics_75["mean_iou"]),
            "tp": int(metrics_75["tp"]),
            "fp": int(metrics_75["fp"]),
            "fn": int(metrics_75["fn"]),
        },
        "total_ground_truth": int(total_gt),
        "total_detections": int(total_detections),
    },
    "yolo_official_metrics": {
        "map50": float(map50),
        "map50_95": float(map5095),
    },
    "hardware_suitability": {
        "lightweight": lightweight,
        "fast": fast if 'avg_inference_time' in globals() else None,
        "accurate": accurate,
    },
}

os.makedirs(f"{WORK_DIR}/results", exist_ok=True)
with open(f"{WORK_DIR}/results/evaluation_metrics_yolov8s.json", "w") as f:
    json.dump(metrics_data, f, indent=4)

print("Metrics saved to results/evaluation_metrics_yolov8s.json")
print()
print("ALL PHASES COMPLETE (YOLOv8s)")


PHASE 5: CALCULATE METRICS (YOLOv8s)

Calculating custom metrics at IoU=0.50 and IoU=0.75...

CUSTOM METRICS (from Phase 4 detections)
----------------------------------------
IoU=0.50 -> P: 0.5866, R: 0.5516, F1: 0.5686, mean IoU(TP): 0.7818
IoU=0.75 -> P: 0.3789, R: 0.3563, F1: 0.3673, mean IoU(TP): 0.8676

True Positives (IoU=0.50): 935
False Positives (IoU=0.50): 659
False Negatives (IoU=0.50): 760
Total detections:          1594
Total ground-truth boxes:  1695

Running official YOLOv8 validation on the val split for mAP...
Ultralytics 8.3.234 🚀 Python-3.12.9 torch-2.9.1+cu128 CPU (Intel Xeon CPU E5-2680 v4 @ 2.40GHz)
val: Fast image access ✅ (ping: 0.4±0.2 ms, read: 319.5±126.5 MB/s, size: 166.2 KB)
val: Scanning /courses/CS5330.202610/students/shinkar.k/Final Project/dataset/labels/val.cache... 107 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 107/107 315.6Kit/s 0.0s
val: /courses/CS5330.202610/students/shinkar.k/Final Project/dataset/images/val/ti_lindau_V105_2015sept_100_